In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Decision Trees
A **Decision Tree** is a supervised machine learning algorithm that can be used for both classification and regression tasks. They are a visual representation of the decision-making process. The graph basically looks like an upside-down tree, with the root node at the top, and then it goes down, splitting into different branches and leaves.

Some important terms:

- **Root Node**: The top node of a decision tree from which all branches originate. It's the starting point.
- **Leaf Node**: The end node of a decision tree where no further splitting occurs. They provide the final prediction.
- **Internal Node**: The nodes between the root and leaf nodes. These nodes are used for making decisions and can further split into either leaf nodes or more internal nodes.
- **Branch**: A path from the root to a leaf node. It represents the decisions made to reach a leaf node based on features.
- **Pruning**: The process of trimming unnecessary nodes to optimize the decision tree.

Decision trees break down the data into smaller and smaller subsets via if-else-like conditions, which eventually lead to a leaf node, i.e., a set of predictions. After a decision tree has been implemented, the final tree is also called a set of decision rules. By following these decision rules, predictions on new unseen data can be made.

#### CART Algorithm

There are two prominent algorithms that utilize decision trees for classification tasks: **CART (Classification and Regression Trees)** and **ID3 (Iterative Dichotomiser 3)**. We will first explore CART specifically for classification tasks, then discuss CART for regression tasks, and finally examine ID3.


How does CART Perform the Split :
CART follows a *greedy approach*. The algorithm starts with the entire dataset and calculates the best way to split the data. It selects the best split by minimizing the **Gini impurity** (aka GINI Index or Metric) or finding the maximum value of **Information Gain**.

**Gini Index (or Metric):** The Gini index measures the probability of a random instance being misclassified when chosen randomly. In other words, it measures how mixed or impure the data is (if the data is highly impure, the chances of picking a random instance and it being misclassified are high). Its value ranges from 0 to 1, where 0 indicates a pure dataset and 1 indicates an impure dataset.

The formula goes like :
For a node $ t $ with $ m $ classes, the Gini impurity $ G(t) $ is calculated as:

$$
G(t) = 1 - \sum_{i=1}^{m} p_i^2
$$

Where:
- $ p_i $ is the proportion of instances in the node that belong to class $ i $.
- $ m $ is the total number of classes.

**Information Gain:** It measures the increase in information from one state to another. Specifically, it checks if splitting on a particular feature will yield a higher information gain. It does this by subtracting the weighted average of the Gini impurity of all subsets created after splitting the tree from the Gini impurity of the parent node. In other words, it checks if the data becomes more pure or impure after splitting. If the data becomes more pure, the split is considered beneficial and is performed.

$$
\text{Information Gain (Gini)} = \text{Gini}(S) - \text{Gini}(S, A)
$$

Were:
- $\text{Gini}(S)$ is the Gini impurity of the original dataset
- $\text{Gini}(S, A)$ is the weighted average Gini impurity of the subsets created by splitting on feature $A$.


The split is performed by asking binary questions like "Does this instance belong to this class?" with answers being either YES or NO. Thus, CART with the Gini index performs a binary split on the data. It starts by calculating the Gini impurity of the entire dataset. Then, it performs splits using all the different features. For each split, it calculates the Gini impurity of the resulting child nodes and computes their weighted average. The Algortihm then calculates the information gain. Then the split that results in the highest Information Gain is selected and performed.

This splitting process occurs recursively until the data becomes pure (i.e., the Gini impurity of a node reaches 0 or informatoin gain becomes 0) for all the nodes that are created (or a stopping criterior is reached such as max tree depth). These final nodes, where the data is *pure*, represent the final predictions for specific inputs.

The **Information Gain** we just read is Also Performed using something called **Entropy**, more on this later.

#### Let's implement a Basic Decision Tree



In [2]:
data = [
    ['Rainy', 'Hot', 'High', 'f', 'no'],
    ['Rainy', 'Hot', 'High', 't', 'no'],
    ['Overcast', 'Hot', 'High', 'f', 'yes'],
    ['Sunny', 'Mild', 'High', 'f', 'yes'],
    ['Sunny', 'Cool', 'Normal', 'f', 'yes'],
    ['Sunny', 'Cool', 'Normal', 't', 'no'],
    ['Overcast', 'Cool', 'Normal', 't', 'yes'],
    ['Rainy', 'Mild', 'High', 'f', 'no'],
    ['Rainy', 'Cool', 'Normal', 'f', 'yes'],
    ['Sunny', 'Mild', 'Normal', 'f', 'yes'],
    ['Rainy', 'Mild', 'Normal', 't', 'yes'],
    ['Overcast', 'Mild', 'High', 't', 'yes'],
    ['Overcast', 'Hot', 'Normal', 'f', 'yes'],
    ['Sunny', 'Mild', 'High', 't', 'no']
]

dataset = pd.DataFrame(data)
dataset.columns = ['Outlook','Temp','Humidity','Windy','Play']
# Mapping dictionary
mapping = {'yes': 1, 'no': 0}
# Convert the 'Play' column
dataset['Play'] = dataset['Play'].map(mapping)

X = dataset.drop('Play',axis = 1)

y = dataset['Play']

print("Input values:\n",X)
print("Output values:\n",y)
print(dataset)

Input values:
      Outlook  Temp Humidity Windy
0      Rainy   Hot     High     f
1      Rainy   Hot     High     t
2   Overcast   Hot     High     f
3      Sunny  Mild     High     f
4      Sunny  Cool   Normal     f
5      Sunny  Cool   Normal     t
6   Overcast  Cool   Normal     t
7      Rainy  Mild     High     f
8      Rainy  Cool   Normal     f
9      Sunny  Mild   Normal     f
10     Rainy  Mild   Normal     t
11  Overcast  Mild     High     t
12  Overcast   Hot   Normal     f
13     Sunny  Mild     High     t
Output values:
 0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Play, dtype: int64
     Outlook  Temp Humidity Windy  Play
0      Rainy   Hot     High     f     0
1      Rainy   Hot     High     t     0
2   Overcast   Hot     High     f     1
3      Sunny  Mild     High     f     1
4      Sunny  Cool   Normal     f     1
5      Sunny  Cool   Normal     t     0
6   Overcast  Cool   Normal     t     1
7 

###### Looks Familier right ?
#### First, we need to implement the GINI and Inforamtion Gain Calculator.


In [3]:
def gini_index_calc(Y):
    """
    This function takes the data of a given node and split it according to GINI index.

    Args:
        Y (ndarray) : target values

    returns:
        gini_impurity (scalar) : GINI index value
    """
    unique_target_vals = np.unique(Y)
    gini_sum = 0

    for cls in unique_target_vals:
        gini_sum += (len(Y[Y == cls])/len(Y))**2

    gini_impurity = 1 - gini_sum
    return gini_impurity



def information_gain(parent, l_child, r_child):
    """
    This function computes the Information Gain for a particluar split
    
    Args:
        parent (ndarray) : Y values of Parent Node
        l_child (ndarray) : Y values of Left Child Node
        r_child (ndarray) : Y values of right Child Node

    Returns:
        gain_value (float) : Information Gain
    """
    gain_value = gini_index_calc(parent)
    
    weight_left = len(l_child) / len(parent)
    weight_right = len(r_child) / len(parent)
    
    left_gini = gini_index_calc(l_child)
    right_gini = gini_index_calc(r_child)
    
    gain_value -= (weight_left * left_gini + weight_right * right_gini)

    return gain_value,left_gini,right_gini



#### Now, we will implement the Split dataset function and function to find the best split.

In [4]:
def split_dataset(data,feature,value):
    """
    Splits the dataset according to the feature and it's value

    Args:
        data (pd.dataframe) : input data
        feature (string) : feature to split with
        value (string) : specific value in that feature to split with

    Returns:
        left_split (pd.dataframe) : left split of dataset
        right_split (pd.dataframe) : right split of dataset
    """
    # Left split
    left_split = data[(data[feature] == value)]
    
    # Right split
    right_split = data[(data[feature] != value)]
    
    return left_split, right_split

def find_best_split(data):
    """
    Finds the best feature to split the dataset

    Args:
        data (pd.dataframe) : dataset
        
    Returns:
        best_split (Dict) : dictionari containg all the relevant info about the best feature to split with
    """

    best_split = {}
    best_split['info_gain'] = 0
    features = data.columns

    target = data.columns[-1]

    for feature in features:
        #skip the iteration where the feature is equal to the target value
        if feature == target:
            continue

        feature_values = data[feature].unique()
 
        for feature_value in feature_values:
            
            parent_y = data[target].values
            
            #spliting the dataset in to nodes to find info gain from the split
            left_split,right_split = split_dataset(data,feature,feature_value)
            
            #left split is the case of feature_value, we will now extract the target value in this case
            y_left = left_split[target].values 
            
            #right split is the case all the other values
            y_right = right_split[target].values
            

            info_gain,left_gini,right_gini = information_gain(parent_y,y_left,y_right) #this will give the info gain of the current split
            #print(info_gain,feature,feature_value)
            if info_gain > best_split['info_gain'] :
                best_split['info_gain'] = info_gain
                best_split['feature'] = feature
                best_split['value'] = feature_value
                best_split['left_gini'] = left_gini #for understanding how leaf node is made
                best_split['right_gini'] = right_gini
                

    return best_split

    

In [5]:
#testing our function are working properly : the output should be Overcast.
print(find_best_split(dataset))

{'info_gain': 0.10204081632653056, 'feature': 'Outlook', 'value': 'Overcast', 'left_gini': 0.0, 'right_gini': 0.5}


#### Now, let's implement the Algo for running this in recursion and building the tree

In [6]:
#first we need to create a TreeNode with a left and right node
class TreeNode:
    def __init__(self, feature=None, value=None, left=None, right=None, is_leaf=False, prediction=None):
        """
        Constructor Function
        """
        self.feature = feature        # Feature to split on
        self.value = value            # Feature's Value to split on
        self.left = left              # Left child
        self.right = right            # Right child
        self.is_leaf = is_leaf        # Whether this node is a leaf node
        self.prediction = prediction  # Prediction if this node is a leaf

In [7]:
def build_tree(data, depth=0, max_depth=None):
    """
    Recursively builds a decision tree from the dataset.
    
    Args:
        data (pd.DataFrame): The dataset with features and target.
        depth (int): Current depth of the tree.
        max_depth (int): Maximum depth of the tree.
    
    Returns:
        TreeNode: The root node of the decision tree.
    """
    
    # Extracting target column
    target = data.columns[-1]
    
    # if all examples have the same label (this basically checks if the gini is 0)
    if len(data[target].unique()) == 1:
        return TreeNode(is_leaf=True, prediction=data[target].iloc[0])
    
    # if there are no features left to split on or if max depth is reached (this checks the constraints)
    if len(data.columns) == 1 or (max_depth is not None and depth >= max_depth):
        majority_class = data[target].mode()[0]
        return TreeNode(is_leaf=True, prediction=majority_class)
    
    # Find the best split
    best_split = find_best_split(data)

    # Split the data
    left_split, right_split = split_dataset(data, best_split['feature'], best_split['value'])
    
    # Recursively build the left and right subtrees
    left_node = build_tree(left_split, depth + 1, max_depth)
    right_node = build_tree(right_split, depth + 1, max_depth)
    
    # Return the decision tree node
    return TreeNode(
        feature=best_split['feature'],
        value=best_split['value'],
        left=left_node,
        right=right_node
    )


In [8]:
# Defining the Root Node and Building the Tree
tree_root = build_tree(dataset)


#Function to Print the Tree
def print_tree(node, depth=0):
    """
    Recursively prints the decision tree.
    
    Args:
        node (TreeNode): The current node in the tree.
        depth (int): The current depth of the node in the tree.
    """
    # Print the current node's details
    if node.is_leaf:
        print(f"{'  ' * 2 * depth}Leaf: Predict {node.prediction}")
    else:
        print(f"{'  ' * 2 * depth}Node: Split on '{node.feature}' <==== {node.value}")
        print(f"{'  ' * 2 * (depth + 1)}Left:")
        print_tree(node.left, depth + 1)
        print(f"{'  ' * 2 *(depth + 1)}Right:")
        print_tree(node.right, depth + 1)


print_tree(tree_root)

Node: Split on 'Outlook' <==== Overcast
    Left:
    Leaf: Predict 1
    Right:
    Node: Split on 'Humidity' <==== High
        Left:
        Node: Split on 'Outlook' <==== Rainy
            Left:
            Leaf: Predict 0
            Right:
            Node: Split on 'Windy' <==== f
                Left:
                Leaf: Predict 1
                Right:
                Leaf: Predict 0
        Right:
        Node: Split on 'Windy' <==== f
            Left:
            Leaf: Predict 1
            Right:
            Node: Split on 'Outlook' <==== Sunny
                Left:
                Leaf: Predict 0
                Right:
                Leaf: Predict 1


#### To visualize the decision tree effectively, we can use libraries like graphviz and sklearn. 

In [9]:
from sklearn import tree
import graphviz

# Define the dataset
data = [
    ['Rainy', 'Hot', 'High', 'f', 'no'],
    ['Rainy', 'Hot', 'High', 't', 'no'],
    ['Overcast', 'Hot', 'High', 'f', 'yes'],
    ['Sunny', 'Mild', 'High', 'f', 'yes'],
    ['Sunny', 'Cool', 'Normal', 'f', 'yes'],
    ['Sunny', 'Cool', 'Normal', 't', 'no'],
    ['Overcast', 'Cool', 'Normal', 't', 'yes'],
    ['Rainy', 'Mild', 'High', 'f', 'no'],
    ['Rainy', 'Cool', 'Normal', 'f', 'yes'],
    ['Sunny', 'Mild', 'Normal', 'f', 'yes'],
    ['Rainy', 'Mild', 'Normal', 't', 'yes'],
    ['Overcast', 'Mild', 'High', 't', 'yes'],
    ['Overcast', 'Hot', 'Normal', 'f', 'yes'],
    ['Sunny', 'Mild', 'High', 't', 'no']
]


# Convert to DataFrame
df = pd.DataFrame(data, columns=['Outlook', 'Temperature', 'Humidity', 'Windy', 'PlayTennis'])

# Define features and labels
X = df[['Outlook', 'Temperature', 'Humidity', 'Windy']]
y = df['PlayTennis']


In [10]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoders
le_outlook = LabelEncoder()
le_temp = LabelEncoder()
le_humidity = LabelEncoder()
le_windy = LabelEncoder()
le_play = LabelEncoder()

# Fit and transform the data
df['Outlook'] = le_outlook.fit_transform(df['Outlook'])
df['Temperature'] = le_temp.fit_transform(df['Temperature'])
df['Humidity'] = le_humidity.fit_transform(df['Humidity'])
df['Windy'] = le_windy.fit_transform(df['Windy'])
df['PlayTennis'] = le_play.fit_transform(df['PlayTennis'])
print(df)
# Define features and labels again with encoded values
X = df[['Outlook', 'Temperature', 'Humidity', 'Windy']]
y = df['PlayTennis']


    Outlook  Temperature  Humidity  Windy  PlayTennis
0         1            1         0      0           0
1         1            1         0      1           0
2         0            1         0      0           1
3         2            2         0      0           1
4         2            0         1      0           1
5         2            0         1      1           0
6         0            0         1      1           1
7         1            2         0      0           0
8         1            0         1      0           1
9         2            2         1      0           1
10        1            2         1      1           1
11        0            2         0      1           1
12        0            1         1      0           1
13        2            2         0      1           0


In [11]:
# Train the decision tree classifier
clf = tree.DecisionTreeClassifier()
clf.fit(X, y)


DecisionTreeClassifier()

In [12]:
# Export the tree as a DOT file
dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=['Outlook', 'Temperature', 'Humidity', 'Windy'],  
                                class_names=le_play.classes_,  
                                filled=True, rounded=True,  
                                special_characters=True)  

# Create a graph from the DOT data
graph = graphviz.Source(dot_data)  
graph.render("./images/decision_tree", format='png') # Save and display the tree


'images\\decision_tree.png'

![Decision Tree](./images/decision_tree.png)

#### There is a slight difference between my decision tree and the tree created using `sklearn`. In my tree, the final right node is based on the feature "Outlook" with the value "Sunny." However, in the `sklearn` tree, the final right node is based on the feature "Temperature." This is likely due to differences in the implementation of the decision tree algorithm between the two methods.


In [13]:
def predict(node, sample):
    """
    Predict the class label for a given sample.
    
    Args:
        node (TreeNode) : Current TreeNode.
        sample (Input Feature): Dictionary with feature values.
    Returns: 
        Prediction (class label).
    """
    if node.is_leaf:
        return node.prediction
    else:
        feature_value = sample.get(node.feature)
        if feature_value == node.value:
            return predict(node.left, sample) if node.left else None
        else:
            return predict(node.right, sample) if node.right else None


In [14]:
# Making the Prediction
feature_names = ['Outlook', 'Temperature', 'Humidity', 'Windy']
input_features =  ['Sunny', 'Cool', 'Normal', 'f']

sample_dict = dict(zip(feature_names, input_features))

predict(tree_root,sample_dict)

1

#### Understanding Constraints and Prunning

These **Constraints** simply tell the stopping critirea (or base case) for a recusive Tree building Function, few major ones are :
    - Maximum Depth : Prevent the level of tree from increasing than Max depth.
    - Minimum Sample per Split : Fixes the Minimum number of samples that should be present on a node for making a split.
    - Minimum Sample per leaf : Fixes the Number of samples required to be at a leaf.
    These help in preventing the Tree from Overfitting on Data, impoves it's generalization and reduces it's complexity.


**Prunning**
It is a Data Compression technique used in Machine Learning and Search Algortihms to reduce the Size of Tree by removing the section of Trees that are not Critical For the classification. It reduces the Complexity of the Tree and helps it not Overfit and generalize better. 

The **Horizen Effect:** Creating a Small Tree might not capture all the effects of different Feature, while a large one might OverFit the data. It is hard to tell when a tree algorithm should stop because it is impossible to tell if the addition of a single extra node will dramatically decrease error. This problem is known as the horizon effect.  

It is dues to this Prunning and it's related techniques are used.

Prunning Techniques :

- **Pre-Prunning:** These are the Stopping Criterion or the Constraints, that tries to keep the Tree Small from the beggining.
- **Post-Prunning:** is the most comman way, here we start with a complex tree then it's subtrees and nodes are replaced with leafs (given that it doesn't significanlty affect the accuracy of the tree). It decreases the Size of the Tree Significantly and even increases Accuracy over unseen data.

Prunning Algorithms :

**Reduced Error Pruning**

One of the simplest forms of pruning (post-pruning). Here, first, the entire tree is grown to its full extent, and the tree almost perfectly fits the data. A validation set is prepared, and in a bottom-up way, we traverse each node and replace the subsequent subtree with a single node consisting of the most popular class. The accuracy is then calculated, and if it doesn't decrease from the original accuracy, this change is kept. This results in a simpler and more generalized tree.

A few important points:

- Choice of Validation Set: The validation set should be a good representation of the overall data.
- Too Much Pruning Can Lead to Underfitting: Excessive pruning can cause the model to become too simplistic, failing to capture important patterns in the data.


**Cost Complexity Pruning**

Cost complexity pruning is a method used to prevent overfitting by balancing the complexity of the decision tree with its accuracy. Similar to Reduced Error Pruning (REP), the criteria for selecting a subtree and replacing it with a node are different. In this method, a measure called cost complexity is calculated. If its value doesn't increase significantly, then the change is kept. This is done in a bottom-up fashion, where every non-leaf node is a candidate for change. The change made is that the non-leaf node and its subtree are replaced with a leaf node representing the majority class of that node. This process is repeated iteratively until only the root node is left. Finally, using a cross-validation set, the tree with the lowest error is considered the best and is selected.

The Formula for the Error calculation goes like:

$$
\text{Cost}(T) = \text{Err}(T) + \alpha \times |T|
$$

where:
- $\text{Err}(T)$ is the proportion of samples in the dataset \( S \) that are incorrectly classified by the tree \( T \) (further this can also be based on the Loss functions -- GINI, Entropy).
- $\alpha$ is a hyperparameter that weights the complexity penalty (sort of like the learning rate).
- $|T|$ represents the number of leaf nodes in the tree.

Wikipedia Defines this process a little more Procedurally as:

**Start with a Fully Grown Tree ($ T_0 $)**:

Begin with the initial decision tree, $ T_0 $, which is fully grown and overfits the training data.

**Generate a Sequence of Trees**:

Create a sequence of trees $ T_0, T_1, \ldots, T_m $, where each subsequent tree $ T_i $ is a pruned version of the previous tree $ T_{i-1} $. The sequence progresses from the fully grown tree to a tree with only the root node.

**Pruning Step-by-Step**:

At each iteration, consider removing a subtree $ t $ from the current tree $ T_{i-1} $.  
Replace this subtree with a leaf node that predicts the most frequent class among the samples covered by the subtree.

**Choosing Which Subtree to Prune**:

Evaluate each potential subtree for removal by calculating the error rate difference:

$$
\operatorname{err}(\operatorname{prune}(T_{i-1}, t), S) - \operatorname{err}(T_{i-1}, S)
$$

Normalize this difference by the change in the number of leaf nodes to determine the cost complexity:

$$
\frac{\operatorname{err}(\operatorname{prune}(T_{i-1}, t), S) - \operatorname{err}(T_{i-1}, S)}{\left| \operatorname{leaves}(T_{i-1}) \right| - \left| \operatorname{leaves}(\operatorname{prune}(T_{i-1}, t)) \right|}
$$

Select the subtree that results in the smallest increase in error per leaf node removed, effectively balancing the trade-off between simplicity and accuracy.

**Pruning Functionality**:

Use the function $ \operatorname{prune}(T, t) $ to systematically remove the chosen subtree from the tree $ T $.

**Final Tree Selection**:

After generating the sequence of pruned trees, evaluate the accuracy of each tree using a validation set or cross-validation.  
The best tree is the one that provides the optimal balance between tree size (complexity) and predictive accuracy on the validation data.

Note: The initial Set \( S \) that is used to calculate error in both is usually the training set, and the final selection of the tree is done on a separately prepared validation or cross-validation set.

---


#### ID3 (Iterative Dichotomizer 3)

Typically used for natural language processing, ID3 is a decision tree-based classification algorithm. Similar to CART, it uses entropy as the loss measure instead of GINI. The overall process of building the tree is similar, involving iterations through possible splits and selecting the one with the highest information gain or lowest entropy.

**Entropy:** Entropy measures the uncertainty in the data. Specifically, it represents the amount of uncertainty or impurity when predicting the class of a randomly chosen sample from the dataset. Higher entropy indicates greater uncertainty about the class of the sample.

$$
H(S) = - \sum_{i=1}^{n} p_i \log_2(p_i)
$$

where:
- $S$ is the dataset.
- $n$ is the number of classes in the dataset.
- $p_i$ is the proportion of samples belonging to class $i$.

**Information Gain (IG):** Information Gain measures how much the entropy is reduced by splitting the dataset on attribute $A$. It is defined as:

$$
IG(S, A) = H(S) - \sum_{v \in \text{Values}(A)} \frac{|S_v|}{|S|} H(S_v)
$$

where:
- $S$ is the original dataset.
- $A$ is the attribute/feature being considered for the split.
- $\text{Values}(A)$ represents the set of all possible values of attribute/feature $A$.
- $S_v$ is the subset of dataset $D$ where attribute $A$ has value $v$.
- $|S_v|$ is the number of samples in subset $S_v$.
- $|S|$ is the number of samples in the original dataset $S$.
- $H(S)$ is the entropy of the original dataset.
- $H(S_v)$ is the entropy of the subset $S_v$.

Here, Information Gain measures how much the entropy is reduced by splitting the dataset on feature $A$. A higher Information Gain indicates a more informative feature for classification, meaning that this feature or features reduce the uncertainty discussed above.


In [15]:
# Entropy Calculation
def entropy(y):
    """
    This function calculates the entropy of a given dataset.

    Args: 
        y (ndarray): Array of target values.
    Returns:
        entropy (float): Entropy value.
    """

    class_labels = np.unique(y)
    entropy = 0
    for cls in class_labels:
        p_cls = len(y[y == cls]) / len(y)  # Calculating the probability of a particular class
        if p_cls > 0:  # Check to avoid log(0)
            entropy += - (p_cls * np.log2(p_cls)) # taking the log, Summing the probabilities
    return entropy

# Information Gain Calculation
def information_gain_entropy(parent, l_child, r_child):
    """
    This function computes the Information Gain for a particular split.
    
    Args:
        parent (ndarray): Y values of the parent node.
        l_child (ndarray): Y values of the left child node.
        r_child (ndarray): Y values of the right child node.

    Returns:
        gain_value (float): Information Gain.
    """
    # Compute entropy of the parent node
    parent_entropy = entropy(parent)
    
    # Compute the weights of each child node
    weight_left = len(l_child) / len(parent)
    weight_right = len(r_child) / len(parent)
    
    # Compute entropy of the child nodes
    left_entropy = entropy(l_child)
    right_entropy = entropy(r_child)
    
    # Calculate information gain
    gain_value = parent_entropy - (weight_left * left_entropy + weight_right * right_entropy)

    return gain_value, left_entropy, right_entropy

The functions created previously can be further generalized to use different loss functions by using a string argument to specify whether to use GINI or Entropy.

#### With the basic classification algorithms in decision trees covered, the next step will be to explore the Decision Tree Regressor. Following that, I'll dive into more advanced algorithms and techniques to enhance decision trees, such as Random Forest Classifier, Ensemble Trees, XGBoost, AdaBoost, and more.
